In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.listdir('/content/drive/My Drive/')

Mounted at /content/drive


['Mane_Minasyan_Assignment group functioning - individual part 2023.docx',
 'Report Group 6.gdoc',
 'Meeting reports',
 'Protocol Elements ROSE study_2024.gdoc',
 'ROSE STUDY GROUP 17 SLIDES_group17.pptx',
 'Tree.html',
 'Data Visualization-Visual 1 demo.mov',
 'Demo-visual1.mov',
 'group34_report_design.gdoc',
 'Kaggle',
 'Colab Notebooks',
 'UHASSELT_signed.pdf',
 '1_Spark_Overview.ipynb',
 '2_Exercises.ipynb',
 '1_Spark_Overview (1).ipynb',
 '3_Dataframe_Overview.ipynb',
 '2_Exercises_filled_ManeMinasyan.ipynb',
 '4_Exercises_Dataframe_filled_ManeMinasyan.ipynb',
 'tutorial.ipynb',
 'tutorial2_filled_ManeMinasyan.ipynb',
 'ANN_&_DL_2425_Homework_ManeMinasyan.ipynb',
 'world_happiness_2015.csv',
 'dimensionality_reduction_countries_ManeMinasyan.ipynb',
 'Digital Epidemology',
 'Deep Learning Project']

In [5]:
selected_columns = ["COUNTRY_CODE", "ISO_WEEKSTARTDATE", "ISO_YEAR", "ISO_WEEK", "INF_A", "INF_B","INF_ALL",
                    "ILI_ACTIVITY", "SPEC_PROCESSED_NB"]



*   INF_A :The number of Influenza A detections (all subtypes) reported for a given week and country
*   INF_B : The number of Influenza B detections (all subtypes) reported for a given week and country

* ILI_ACTIVITY: Influenza-like illness reported relative number

In [6]:
data_flu = pd.read_csv("/content/drive/My Drive/Digital Epidemology/data/flu_data.csv")
data_flu_selected = data_flu[selected_columns]
data_flu_nl = data_flu_selected[data_flu_selected["COUNTRY_CODE"] == "NLD"]

In [7]:
df_google_serach = pd.read_csv("/content/drive/My Drive/Digital Epidemology/data/query_search_netherlands.csv")

In [8]:
weather_nl = pd.read_csv("/content/drive/My Drive/Digital Epidemology/data/Dutch_weather.csv", low_memory=False)
weather_nl = weather_nl.dropna()
weather_nl['Date'] = pd.to_datetime(weather_nl['YYYYMMDD'], format='%Y%m%d')

In [9]:
weather_nl.rename(columns={
    "FHVEC": "wind_speed",
    "FHX": "hourly_highest",
    "FHN": "hourly_lowest",
    "TG": "24_avg_temp",
    "SQ": "sunshine_duration",
    "RH": "24_precipitation",
    "NG": "24_avg_cloud_cover",
    "UG": "24_avg_humidity",
    "EV24": "evaporation",
    "DATE": "DATE"
}, inplace=True)

In [10]:
data_flu_nl = data_flu_nl.copy()
data_flu_nl["ISO_WEEKSTARTDATE"] = pd.to_datetime(data_flu_nl["ISO_WEEKSTARTDATE"], errors="coerce")
data_flu_nl["Year-Month"] = data_flu_nl["ISO_WEEKSTARTDATE"].dt.to_period("M")

In [11]:
weather_nl = weather_nl.copy()
weather_nl["Year-Month"] = weather_nl["Date"].dt.to_period("M")
weather_monthly = weather_nl.groupby("Year-Month").mean(numeric_only=True).reset_index()

In [12]:
flu_cases_monthly = (
    data_flu_nl
    .drop(columns=["ISO_WEEKSTARTDATE"])
    .groupby("Year-Month")
    .sum(numeric_only=True)
    .reset_index()
)

In [13]:
df_google_serach["Year-Month"] = pd.to_datetime(df_google_serach["Year-Month"], errors="coerce")
df_google_serach["Year-Month"] = df_google_serach["Year-Month"].dt.to_period("M")

In [14]:
merged_df = pd.merge(df_google_serach, weather_monthly, on="Year-Month", how="inner")
merged_df_final = pd.merge(merged_df, flu_cases_monthly, on="Year-Month", how="inner")

In [15]:
selected_columns_flu = ["wind_speed","hourly_highest","hourly_lowest","24_avg_temp","sunshine_duration","24_precipitation","24_avg_humidity",
                        "evaporation","Year-Month", "flu: (Netherlands)", "Influenza: (Netherlands)",
       "Griep: (Netherlands)", "Total Search", "ISO_YEAR","ISO_WEEK","INF_A", "INF_B","INF_ALL" ,"ILI_ACTIVITY", "SPEC_PROCESSED_NB"]
merged_df_final_selected = merged_df_final[selected_columns_flu]

In [16]:
merged_df_final_selected.to_csv("/content/drive/My Drive/Digital Epidemology/data/flu_data_complete_new.csv")